In [105]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [106]:
from torch.utils.data import DataLoader, Dataset, random_split
import torch
import torch.nn as nn
from torchvision.io import read_image
import torchvision.transforms as transforms
from torchvision import datasets
from torch import optim
import os

In [107]:
transform = transforms.Compose([
    transforms.Resize((240, 240)),  # Resize the images to a fixed size
    transforms.ToTensor()           # Convert PIL images to tensors
])

In [108]:
dataset = datasets.ImageFolder("data/", transform=transform)
train_size = int(0.7 * len(dataset))
valid_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [109]:
train_dataset, remaining = random_split(dataset, [train_size, len(dataset)-train_size])
valid_dataset, test_dataset = random_split(remaining, [val_size, test_size])

In [110]:
import torch
from torchvision import transforms

batch_size=32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

In [111]:
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3*240*240, 2),
    nn.LogSigmoid()
).cuda()

In [112]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [113]:
def train(dl, model, loss_fn, opt):
    size = len(dl.dataset)
    model.train()
    for batch, (image, label) in enumerate(dl):
        image = image.to("cuda")
        label = label.to("cuda")
        pred = model(image)
        loss = loss_fn(pred, label)
        loss.backward()
        opt.step()
        opt.zero_grad()

        if batch % 100==0:
            print(f"loss: {loss.item()}    [{(batch+1)*len(image)}/{size}]")

In [114]:
train(train_loader, net, loss_fn, optimizer)

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [115]:
def evaluate(dl, model, loss_fn, opt):
    size = len(dl.dataset)
    num_batches = len(dl)
    model.eval()
    test_loss, total_correct = 0, 0
    with torch.no_grad():
        for image, label in dl:
            image, label = image.to(device), label.to(device)
            pred = model(image)
            loss = loss_fn(pred, label)
            test_loss += loss.item()
            correct = (pred.argmax(1) == label).sum().item()
            total_correct += correct        
    average_loss = test_loss / num_batches
    accuracy = total_correct / size
    print(f"Accuracy = {(100*accuracy):>0.1f}%, Avg_Loss = {average_loss}")

In [116]:
for epoch in range(5):
    print(f"====== Epoch {epoch+1} ======")
    train(train_loader, net, loss_fn, optimizer)
    evaluate(val_loader, net, loss_fn, optimizer)
print("Done!")

====== Epoch 1 ======


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>